In [242]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [243]:
stories_json = []
with open('./output_stories_0122.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

86
dict_keys(['index', 'stories_index', 'title', 'broadcasting_date', 'abstracts_list'])


In [244]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined
0,0700,0,「からだねん土でスマートになろう」,2022年4月2日,空き地で筋肉（きんにく）じまんをしていたジャイアンから、うでに筋肉がまったくないことをバカに...,空 き 地 で 筋 肉 （ き ん に く ） じ ま ん を し て い た ジ ャ イ ...
1,0700,1,「ナゲーなげなわ」,2022年4月2日,部屋で寝（ね）転びながらマンガを読み、自分は寝ころんだままでジュースのおかわりをたのんでくる...,部 屋 で 寝 （ ね ） 転 び な が ら マ ン ガ を 読 み 、 自 分 は 寝 ...
2,0701,0,「いただき小ばん」,2022年4月9日,のび太から歩かないでどこへでも行ける道具はないかと聞かれたドラえもんは、『いただき小ばん』を...,の び 太 か ら 歩 か な い で ど こ へ で も 行 け る 道 具 は な い ...
3,0701,1,「どくさいスイッチ」,2022年4月9日,野球の試合で失敗したのび太は、おこったジャイアンに追いかけられ、ひどい目にあってしまう。ドラ...,野 球 の 試 合 で 失 敗 し た の び 太 は 、 お こ っ た ジ ャ イ ア ...
4,0702,0,「のび太のダンボール宇宙ステーション」,2022年4月16日,ダンボールを組み合わせて、庭で“宇宙（うちゅう）ステーションごっこ”をしていたのび太は、本格...,ダ ン ボ ー ル を 組 み 合 わ せ て 、 庭 で “ 宇 宙 （ う ち ゅ う ...


In [245]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [246]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstracts_list"].apply(mecab_tokenizer)
print(sentences)

0     [空き地, 筋肉, 筋肉, じまん, ジャイアン, 筋肉, こと, バカ, のび太, ら, ...
1     [部屋, マンガ, 自分, まま, ジュース, かわり, のび太, ドラえもん, 時, スネ...
2     [のび太, でどこ, 道具, ドラえもん, いただき, 小判, これ, 体, 体, コバンザ...
3     [野球, 試合, 失敗, のび太, ジャイアン, 目, ドラえもん, 目, よう, 練習, ...
4     [ダンボール, 庭, 宇宙, うち, ゅう, ステーション, ごっこ, のび太, 本格, 本...
                            ...                        
81    [星がきれい, 夜, のび太, スネ夫, 家, 前, 庭, 星空, パーティー, 女の子, ...
82    [おなか, 食事, のび太, ご飯, ありがたみ, ママ, のび太, まじめ, 米, パン,...
83    [子供達, 鬼ごっこ, の, のび太, 自分たち, 野郎, みんな, ジャイアン, スネ夫,...
84    [５, 回, 連続, ０, 点, のび太, 来週, テスト, ０, 点, ママ, 鬼, 顔,...
85    [冬, スポーツ, ウインタースポーツ, ランド, のび太, たち, スキー, スケート, ...
Name: abstracts_list, Length: 86, dtype: object


In [247]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [248]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[ 4.303521    3.5863633 ]
 [ 4.7308955   3.6296892 ]
 [ 4.659376    3.209778  ]
 [ 6.023469    4.2472754 ]
 [-0.30414915  0.02889376]
 [ 7.444777    4.897035  ]
 [ 4.733254    3.8989089 ]
 [-2.4394727  -1.7333342 ]
 [ 0.8619304   0.64362544]
 [ 0.06715658  0.26358023]
 [-2.7396693  -2.1669471 ]
 [-2.9698253  -1.8625478 ]
 [ 3.8283155   2.6079488 ]
 [ 7.4375362   4.7830763 ]
 [ 3.5494943   2.860639  ]
 [-1.1430537  -0.65167683]
 [-0.10816781  0.13299024]
 [ 3.130926    2.2350514 ]
 [ 5.2175293   4.2193356 ]
 [ 6.1225457   4.74498   ]
 [-2.0502048  -1.4730786 ]
 [ 1.137834    1.1013503 ]
 [ 1.6219621   1.4480535 ]
 [ 1.1608485   0.8277604 ]
 [ 2.8661351   2.0712504 ]
 [ 2.0025852   1.5914505 ]
 [-2.9981768  -2.234132  ]
 [ 0.78728145  0.8690659 ]
 [-1.1823655  -0.9582554 ]
 [ 3.940933    2.6502602 ]
 [-1.8540988  -1.3555461 ]
 [ 7.6623154   5.21796   ]
 [ 2.5962532   2.2089028 ]
 [-2.182505   -1.5645707 ]
 [-1.567103   -1.0818799 ]
 [ 5.5700827   3.9358702 ]
 [ 3.9737883   3.3029592 ]
 

In [249]:
vector_df = pd.DataFrame.from_dict({
    'vector': [v for v in vector_list],
    'vector_x': [v[0] for v in vector_enbedded],
    'vector_y': [v[1] for v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year

In [250]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [251]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()

In [252]:
sampled_df = stories_df_merged.sample(1)
print(sampled_df.head())
sampled_df = sampled_df.rename(columns={"vector": "sampled_vec"})[["sampled_vec"]]


   index  stories_index     title broadcasting_date  \
47  0724              0  「のび太漂流記」        2022年9月17日   

                                       abstracts_list  \
47  みんなから集めたわりばしをならべ、いかだを作って無人島に行くのだと言い出すのび太におどろくド...   

                                      abstract_joined  \
47  み ん な か ら 集 め た わ り ば し を な ら べ 、 い か だ を 作 っ ...   

                                               vector  vector_x  vector_y  \
47  [-0.016062262, 0.008184233, -0.009978594, 0.00...  4.269741  2.376587   

   broadcasting_dt  broadcasting_year  
47      2022-09-17               2022  


In [253]:
stories_df_merged = stories_df_merged.assign(key=1).merge(sampled_df.assign(key=1), on='key').drop('key', axis=1)
stories_df_merged

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined,vector,vector_x,vector_y,broadcasting_dt,broadcasting_year,sampled_vec
0,0700,0,「からだねん土でスマートになろう」,2022年4月2日,空き地で筋肉（きんにく）じまんをしていたジャイアンから、うでに筋肉がまったくないことをバカに...,空 き 地 で 筋 肉 （ き ん に く ） じ ま ん を し て い た ジ ャ イ ...,"[-0.012913099, -0.002905351, -0.014714871, 0.0...",4.303521,3.586363,2022-04-02,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
1,0700,1,「ナゲーなげなわ」,2022年4月2日,部屋で寝（ね）転びながらマンガを読み、自分は寝ころんだままでジュースのおかわりをたのんでくる...,部 屋 で 寝 （ ね ） 転 び な が ら マ ン ガ を 読 み 、 自 分 は 寝 ...,"[-0.012737461, -0.00023994221, -0.010478356, 0...",4.730896,3.629689,2022-04-02,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
2,0701,0,「いただき小ばん」,2022年4月9日,のび太から歩かないでどこへでも行ける道具はないかと聞かれたドラえもんは、『いただき小ばん』を...,の び 太 か ら 歩 か な い で ど こ へ で も 行 け る 道 具 は な い ...,"[-0.014949739, -0.006372355, 0.003940386, -0.0...",4.659376,3.209778,2022-04-09,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
3,0701,1,「どくさいスイッチ」,2022年4月9日,野球の試合で失敗したのび太は、おこったジャイアンに追いかけられ、ひどい目にあってしまう。ドラ...,野 球 の 試 合 で 失 敗 し た の び 太 は 、 お こ っ た ジ ャ イ ア ...,"[-0.009342262, 0.0096354885, -0.0014996169, 0....",6.023469,4.247275,2022-04-09,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
4,0702,0,「のび太のダンボール宇宙ステーション」,2022年4月16日,ダンボールを組み合わせて、庭で“宇宙（うちゅう）ステーションごっこ”をしていたのび太は、本格...,ダ ン ボ ー ル を 組 み 合 わ せ て 、 庭 で “ 宇 宙 （ う ち ゅ う ...,"[-0.022490567, -0.0010356762, -0.01449138, 0.0...",-0.304149,0.028894,2022-04-16,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...
81,0741,1,「夜空に輝くピザ・ギョーザ」,2023年1月14日,星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。女...,星 が き れ い に か が や く 夜 、 の び 太 が ス ネ 夫 の 家 の 前 ...,"[-0.024335511, 0.014852706, -0.016015744, 0.01...",-2.703842,-2.107837,2023-01-14,2023,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
82,0742,0,「ありがたみわかり機」,2023年1月21日,おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。とこ...,お な か が い っ ぱ い だ か ら と 食 事 を 残 す の び 太 に 、 ご ...,"[-0.0132149635, -0.001909591, -0.0068630776, 0...",7.069832,4.618168,2023-01-21,2023,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
83,0742,1,「のび泥棒をタイホせよ！」,2023年1月21日,小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんなを...,小 さ な 子 ど も た ち が お に ご っ こ を し て 遊 ん で い る の ...,"[-0.014431373, 0.007472043, -0.014709043, 0.00...",3.089931,2.399874,2023-01-21,2023,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
84,0743,0,「鬼のパンツはいいパンツ？」,2023年1月28日,５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になってし...,５ 回 連 続 で ０ 点 を 取 っ て し ま っ た の び 太 。 来 週 の テ ...,"[0.0037542062, 0.0031492938, -0.0028632819, -0...",5.875231,4.170030,2023-01-28,2023,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."


In [254]:
stories_df_merged

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined,vector,vector_x,vector_y,broadcasting_dt,broadcasting_year,sampled_vec
0,0700,0,「からだねん土でスマートになろう」,2022年4月2日,空き地で筋肉（きんにく）じまんをしていたジャイアンから、うでに筋肉がまったくないことをバカに...,空 き 地 で 筋 肉 （ き ん に く ） じ ま ん を し て い た ジ ャ イ ...,"[-0.012913099, -0.002905351, -0.014714871, 0.0...",4.303521,3.586363,2022-04-02,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
1,0700,1,「ナゲーなげなわ」,2022年4月2日,部屋で寝（ね）転びながらマンガを読み、自分は寝ころんだままでジュースのおかわりをたのんでくる...,部 屋 で 寝 （ ね ） 転 び な が ら マ ン ガ を 読 み 、 自 分 は 寝 ...,"[-0.012737461, -0.00023994221, -0.010478356, 0...",4.730896,3.629689,2022-04-02,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
2,0701,0,「いただき小ばん」,2022年4月9日,のび太から歩かないでどこへでも行ける道具はないかと聞かれたドラえもんは、『いただき小ばん』を...,の び 太 か ら 歩 か な い で ど こ へ で も 行 け る 道 具 は な い ...,"[-0.014949739, -0.006372355, 0.003940386, -0.0...",4.659376,3.209778,2022-04-09,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
3,0701,1,「どくさいスイッチ」,2022年4月9日,野球の試合で失敗したのび太は、おこったジャイアンに追いかけられ、ひどい目にあってしまう。ドラ...,野 球 の 試 合 で 失 敗 し た の び 太 は 、 お こ っ た ジ ャ イ ア ...,"[-0.009342262, 0.0096354885, -0.0014996169, 0....",6.023469,4.247275,2022-04-09,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
4,0702,0,「のび太のダンボール宇宙ステーション」,2022年4月16日,ダンボールを組み合わせて、庭で“宇宙（うちゅう）ステーションごっこ”をしていたのび太は、本格...,ダ ン ボ ー ル を 組 み 合 わ せ て 、 庭 で “ 宇 宙 （ う ち ゅ う ...,"[-0.022490567, -0.0010356762, -0.01449138, 0.0...",-0.304149,0.028894,2022-04-16,2022,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...
81,0741,1,「夜空に輝くピザ・ギョーザ」,2023年1月14日,星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。女...,星 が き れ い に か が や く 夜 、 の び 太 が ス ネ 夫 の 家 の 前 ...,"[-0.024335511, 0.014852706, -0.016015744, 0.01...",-2.703842,-2.107837,2023-01-14,2023,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
82,0742,0,「ありがたみわかり機」,2023年1月21日,おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。とこ...,お な か が い っ ぱ い だ か ら と 食 事 を 残 す の び 太 に 、 ご ...,"[-0.0132149635, -0.001909591, -0.0068630776, 0...",7.069832,4.618168,2023-01-21,2023,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
83,0742,1,「のび泥棒をタイホせよ！」,2023年1月21日,小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんなを...,小 さ な 子 ど も た ち が お に ご っ こ を し て 遊 ん で い る の ...,"[-0.014431373, 0.007472043, -0.014709043, 0.00...",3.089931,2.399874,2023-01-21,2023,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."
84,0743,0,「鬼のパンツはいいパンツ？」,2023年1月28日,５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になってし...,５ 回 連 続 で ０ 点 を 取 っ て し ま っ た の び 太 。 来 週 の テ ...,"[0.0037542062, 0.0031492938, -0.0028632819, -0...",5.875231,4.170030,2023-01-28,2023,"[-0.016062262, 0.008184233, -0.009978594, 0.00..."


In [257]:
from scipy.spatial.distance import cosine

stories_df_merged["cosine_similarity"] =  stories_df_merged.apply(lambda v: 1 - cosine(v["vector"], v["sampled_vec"]), axis=1)
stories_df_merged_sort = stories_df_merged[["title", "abstract_joined", "cosine_similarity"]].sort_values("cosine_similarity", ascending=False)


In [258]:
stories_df_merged_sort

,title,abstract_joined,cosine_similarity
47,「のび太漂流記」,み ん な か ら 集 め た わ り ば し を な ら べ 、 い か だ を 作 っ ...,1.000000
52,「大富豪のび太」,手 に 取 っ た マ ン ガ 本 か ら 一 万 円 札 を 発 見 し た の び 太 ...,0.951623
72,「うちでのデパート」,マ マ が ド ラ マ を 見 る た め 、 楽 し み に し て い る ア ニ メ ...,0.951239
26,「海坊主がつれた！」,た く さ ん の 魚 を 釣 （ つ ） っ て き た と い う ス ネ 夫 か ら ...,0.950396
71,「バトンタッチでおまかせ！自動返送荷札」,ス ネ 夫 が 持 っ て い る 人 気 の マ ン ガ 本 を 早 く 借 り た い ...,0.948526
...,...,...,...
65,「宇宙戦艦のび太を襲う」,し ず か や 出 木 杉 か ら 、 人 間 は ふ し ぎ な 予 知 夢 （ よ ち ...,0.691053
44,「勝利を呼ぶチアリーダー手ぶくろ」,ス ネ 夫 と オ セ ロ 対 決 を し た の び 太 は 、 と ち ゅ う ま で ...,0.661882
43,「さがせ！野比家のへっぽこご先祖さま」,ス ネ 夫 の 部 屋 で 、 ジ ャ イ ア ン と ス ネ 夫 が 最 新 の V R ...,0.659474
31,「風神さわぎ」,節 電 の た め に エ ア コ ン の ス イ ッ チ を 切 っ て し ま う マ ...,0.450483
